# 嵌入示例

## 准备工作

In [1]:
!pip install openai FlagEmbedding

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.3/312.3 KB 7.1 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.4/140.4 KB 7.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 KB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.3/409.3 KB 10.9 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 KB 9.0 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 KB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 KB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 KB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 7.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 7.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/11

## 最简单的嵌入

In [2]:
%%time

from FlagEmbedding import FlagModel

sentences_1 = ["样例数据-1", "样例数据-2"]
sentences_2 = ["样例数据-3", "样例数据-4"]

model = FlagModel('/models/bge-large-zh-v1.5', 
                  query_instruction_for_retrieval="为这个句子生成表示以用于检索相关文章：",
                  use_fp16=True) # Setting use_fp16 to True speeds up computation with a slight performance degradation

embeddings_1 = model.encode(sentences_1)
print(embeddings_1)
embeddings_2 = model.encode(sentences_2)
print(embeddings_2)

similarity = embeddings_1 @ embeddings_2.T
print(similarity)
print('\n')

# for s2p(short query to long passage) retrieval task, suggest to use encode_queries() which will automatically add the instruction to each query
# corpus in retrieval task can still use encode() or encode_corpus(), since they don't need instruction
queries = ['query_1', 'query_2']
passages = ["样例文档-1", "样例文档-2"]
q_embeddings = model.encode_queries(queries)
p_embeddings = model.encode(passages)
scores = q_embeddings @ p_embeddings.T
print(scores)
print('\n')

# 测试
queries = ['嵌入向量之间的相似度是0.8，算高么', '什么是素数', '质数的定义是啥']
p1="""
嵌入向量之间的相似度为0.8通常可以被认为是相当高的。在许多情况下，嵌入向量之间的相似度在0到1之间，1表示完全相似，0表示完全不同。因此，0.8的相似度意味着这两个向量之间存在着相当强的相似性。不过，具体的评价会取决于你的应用场景以及对相似度的要求。
"""
p2="""
质数是指大于1的整数，它除了1和它本身外没有其他正因数。换句话说，质数只能被1和自身整除，不能被其他数字整除。例如，2、3、5、7、11等都是质数，因为它们只能被1和自身整除。而4、6、8、9等不是质数，因为它们可以被除了1和自身以外的其他数字整除。质数在数学中有着重要的地位，它们在数论、密码学等领域有广泛的应用。
"""
passages = [p1,p2]
q_embeddings = model.encode_queries(queries)
p_embeddings = model.encode(passages)
scores = q_embeddings @ p_embeddings.T
print(scores)
print('\n')

[[ 0.001508  0.01648  -0.02812  ... -0.0309    0.02963  -0.03262 ]
 [ 0.01509   0.004154 -0.01569  ... -0.02809   0.0408   -0.02509 ]]
[[ 0.001684 -0.0077   -0.006943 ... -0.0518    0.04968  -0.03397 ]
 [ 0.014336  0.01314  -0.0299   ... -0.02203   0.05545  -0.05283 ]]
[[0.855  0.852 ]
 [0.874  0.8555]]


[[0.3374 0.2048]
 [0.2261 0.3848]]


[[0.811  0.2568]
 [0.3025 0.544 ]
 [0.3022 0.6904]]


CPU times: user 5.35 s, sys: 1.27 s, total: 6.62 s
Wall time: 16.2 s
